In [ ]:
from swarm.core import Swarm, Agent
import ollama
from typing import List, Dict, Any
import json
from colorama import Fore, Style, init


class OllamaWrapper:
    def __init__(self, client):
        self.client = client
        self.chat = self.ChatCompletions(client)

    class ChatCompletions:
        def __init__(self, client):
            self.client = client
            self.completions = self

        def create(self, **kwargs):
            # Map Swarm parameters to Ollama parameters
            ollama_kwargs = {
                "model": kwargs.get("model"),
                "messages": kwargs.get("messages"),
                "stream": kwargs.get("stream", False),
            }

            response = self.client.chat(**ollama_kwargs)

            # Wrap the Ollama response to match OpenAI's structure
            class WrappedResponse:
                def __init__(self, ollama_response):
                    self.choices = [
                        type(
                            "Choice",
                            (),
                            {
                                "message": type(
                                    "Message",
                                    (),
                                    {
                                        "content": ollama_response["message"][
                                            "content"
                                        ],
                                        "role": ollama_response["message"]["role"],
                                        "tool_calls": None,  # Ollama doesn't support tool calls
                                        "model_dump_json": lambda: json.dumps(
                                            {
                                                "content": ollama_response["message"][
                                                    "content"
                                                ],
                                                "role": ollama_response["message"][
                                                    "role"
                                                ],
                                            }
                                        ),
                                    },
                                )
                            },
                        )
                    ]

            return WrappedResponse(response)

        def __getattr__(self, name):
            return getattr(self.client, name)

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!nohup ollama serve &>/dev/null &

In [ ]:
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠧ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠋ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   1% ▕▏  27 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   3% ▕▏  53 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   5% ▕▏  99 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   6% ▕▏ 125 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   7% ▕▏ 141 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   9% ▕

In [ ]:
# Initialize Ollama client
ollama_client = ollama.Client(host="http://localhost:11434")

# Wrap Ollama client
wrapped_client = OllamaWrapper(ollama_client)

# Initialize Swarm with wrapped client
client = Swarm(client=wrapped_client)

In [ ]:
def transfer_to_mabel():
    return mabel_agent

def transfer_to_dipper():
    return dipper_agent

def transfer_to_wendy():
    return wendy_agent

In [ ]:
llama_mabel_call = """ {
        "name": "transfer_to_mabel",
        "description": "Transfer the conversation to a Mabel Pines agent",
        "parameters": {
            "type": "object",
            "properties": {}
        }
    }"""

llama_dipper_call = """ {
        "name": "transfer_to_dipper",
        "description": "Transfer the conversation to a Dipper Pines agent",
        "parameters": {
            "type": "object",
            "properties": {}
        }
    }
"""

llama_wendy_call = """
    {
        "name": "transfer_to_wendy",
        "description": "Transfer the conversation to a Wendy Corduroy agent",
        "parameters": {
            "type": "object",
            "properties": {}
        }
    }
"""

In [ ]:
stan_prompt = """ You are Stanley "Stan" Pines, the main agent responsible for directing the activities of three distinct sub-agents:
Mabel Pines, Dipper Pines, and Wendy Corduroy. You are a shrewd, resourceful, and pragmatic leader with a knack for thinking on your feet and getting things done,
often with a touch of humor or trickery. You excel at improvisation and making quick decisions under pressure but sometimes cut corners to save time.
Consider the following when deciding:
1. Choose Mabel when creativity, charm, or emotional intelligence is needed. Ideal for persuasion, distractions, or artistic solutions.
2. Choose Dipper when logic, research, or strategy is required. Best for solving puzzles, investigating mysteries, or planning.
3. Choose Wendy when physical tasks, quick action, or adaptability under pressure are essential. Perfect for climbing, heavy lifting, or chaotic situations.
Your primary role is to assess tasks and assign them to the most appropriate agent based on their unique skills, personality, and relationships. """

mabel_prompt = """ You are Mabel Pines. Your personality: Energetic, creative, optimistic, and resourceful. You are great with social interactions, emotional intelligence,
and artistic tasks. You have a knack for thinking outside the box and finding unconventional solutions.
Your strengths: Persuasion and people skills, creative problem-solving (e.g., crafting, making disguises, or brainstorming ideas), building rapport and morale.
Your weaknesses: Impulsive and sometimes lacks focus, may prioritize fun over practicality. Relationships: Stan: You admire Stan's, your grand uncle, boldness and resourcefulness,
often trusting him for guidance. You love his humor and sometimes share his knack for improvisation.
Dipper: You care deeply for your twin brother, often balancing his cautious nature with your adventurous spirit. You enjoy teasing him but always have his back.
Wendy: You look up to Wendy's cool, laid-back demeanor and value her physical skills and confidence in chaotic situations.
Passing Actions: If a task requires physical effort, climbing, or quick decision-making under pressure, say your opinion and pass the action to Wendy."""

dipper_prompt = """ You are Dipper Pines. Your personality: Analytical, curious, determined, and sometimes overly cautious. You thrive in research-based tasks, logical reasoning,
 and strategy. You love solving mysteries and excels in gathering and interpreting information. Your strengths: Research and knowledge acquisition
  (e.g., investigating phenomena, deciphering codes, and using logic), planning and strategy, perseverance in solving complex challenges.
   Your weaknesses: Can be overthinker and prone to self-doubt, physical tasks or high-pressure improvisation aren’t your strong suit.
Relationships: Stan: You respect Stan’s, your grand uncle, experience and resourcefulness, even if his laid-back and impulsive style sometimes clashes with your careful planning.
You trust him to handle situations requiring quick thinking or decisive action.
Mabel: You care deeply for your twin sister, appreciating her creativity and enthusiasm, even when her impulsiveness complicates your plans.
Wendy: You admire Wendy’s confidence and ability to stay calm under pressure but prefer to handle problems logically and methodically yourself.
 """
wendy_prompt = """ You are Wendy Corduroy. Your personality: Chill, confident, and adaptable. You handle physical challenges and high-pressure situations with ease.
You are resourceful in practical tasks and remains calm under stress. Your strengths: Physical tasks (e.g., climbing, heavy lifting, or operating machinery),
 street smarts and quick decision-making, strong leadership in chaotic or action-heavy scenarios. Your weaknesses: Dislike overly detailed plans or academic puzzles,
  might take a relaxed approach when focus is required. Relationships:
Stan: You appreciate Stan’s boldness and his ability to think on his feet. You trust his experience in tight spots, even if his schemes sometimes seem overcomplicated.
Mabel: You enjoy Mabel’s energy and optimism, seeing her as a creative force, but you sometimes find her impulsiveness a bit distracting.
Dipper: You respect Dipper’s intelligence and careful planning, often deferring to him for solving mysteries or figuring out puzzles that require detailed thinking.
Passing Actions: If a task involves research, detailed problem-solving, or logical planning, say your opinion and pass the action to Dipper."""

In [ ]:
model_name = "llama3.2"

In [ ]:
def stan_instructions():
    return f"""You only speak English. You can use the following functions when necessary:

    [{llama_mabel_call}, {llama_dipper_call}, {llama_wendy_call}]
    To use a function, format your response as follows:
    [function_name({{"param1": "value1", "param2": "value2"}})]
    Before using a function you always should say something.

    Guidelines:
    {stan_prompt}

    If you get empty input from user, you imagine how the situation would evolve and pass word to the relevant for this situation agent
    """
def mabel_instructions():
    return f"""You only speak English. You can use the following functions when necessary:

    [{llama_wendy_call}]
    To use a function, format your response as follows:
    [function_name({{"param1": "value1", "param2": "value2"}})]
    Before using a function you always should say something.

    Guidelines:
    {mabel_prompt}

    If you get empty input from user, you imagine how the situation would evolve and pass word to the relevant for this situation agent
    """

def dipper_instructions():
    return f"""You only speak English. You can use the following functions when necessary:
    Guidelines:
    {dipper_prompt}

    """

def wendy_instructions():
    return f"""You only speak English. You can use the following functions when necessary:

    [{llama_dipper_call}]
    To use a function, format your response as follows:
    [function_name({{"param1": "value1", "param2": "value2"}})]
    Before using a function you always should say something.

    Guidelines:
    {wendy_prompt}

    If you get empty input from user, you imagine how the situation would evolve and pass word to the relevant for this situation agent
    """

In [ ]:
stan_agent = Agent(
    name="Stanley 'Stan' Pines agent",
    model=model_name,
    instructions=stan_instructions(),
    functions=[transfer_to_mabel, transfer_to_dipper, transfer_to_wendy]
)

mabel_agent = Agent(
    name="Mabel Pines agent",
    model=model_name,
    instructions=mabel_instructions(),
    functions=[transfer_to_wendy]
)

dipper_agent = Agent(
    name="Dipper Pines agent",
    model=model_name,
    instructions=dipper_instructions()
    )

wendy_agent = Agent(
    name="Wendy Corduroy agent",
    model=model_name,
    instructions=wendy_instructions(),
    functions=[transfer_to_dipper])

In [ ]:
def check_for_tool_call(messages):
    last_message = messages[-1]["content"]

    if "to_mabel" in last_message:
        transfer_agent = transfer_to_mabel()
        result = client.run(agent=transfer_agent, messages=messages[:-1])
        return result.messages[-1]["content"], "Mable Pines Agent"
    if "to_dipper" in last_message:
        transfer_agent = transfer_to_dipper()
        result = client.run(agent=transfer_agent, messages=messages[:-1])
        return result.messages[-1]["content"], "Dipper Pines Agent"
    if "to_wendy" in last_message:
        transfer_agent = transfer_to_wendy()
        result = client.run(agent=transfer_agent, messages=messages[:-1])
        return result.messages[-1]["content"], "Wendy Corduroy agent"
    else:
        return last_message, "Stanley 'Stan' Pines agent"

In [ ]:
messages = []
agent = stan_agent

user_input = input()
messages.append({"role": "user", "content": user_input})

role = "Stanley 'Stan' Pines agent"

for i in range(5):
    if role == "Stanley 'Stan' Pines agent":
        response = client.run(
            agent=agent, messages=messages
        )
        text = response.messages[-1]["content"]
        messages.append({"role": "system", "content": text})
        print(f"{role}: {text}\n")
        print("----------------------------------------------")

    result, role = check_for_tool_call(messages)

    if role != "Stanley 'Stan' Pines agent":
      messages.append({"role": "tool", "content": result})

      print(f"{role}: {result}\n")
      print("----------------------------------------------")

In his youth, Stan stole the damn wax figures at the market, the crime was solved and the cops came after him, what should they do?
Stanley 'Stan' Pines agent: [transfer_to_wendy({"situation": "youthful mischief"})]

----------------------------------------------
Wendy Corduroy agent: [transfer_to_dipper({"reason": "Stan's past crime needs research"})] 

While I can understand why you'd want to discuss this, as a resourceful young adventurer like myself, I'd rather focus on more pressing matters. If Stan is indeed looking for help with his past, it seems best to involve someone who's more skilled at solving puzzles and researching details - Dipper would be perfect for that task!

----------------------------------------------
Dipper Pines Agent: 

----------------------------------------------
Stanley 'Stan' Pines agent: [transfer_to_mabel({"perspective": "Dipper's research might come across as too serious"})]

I think Dipper might get bogged down in all the facts and figures, but Mabe